In [1]:
import numpy as np
import bordado as bd

## Gerar dado sintético

$g_z = \frac{Gm(z-z_c)}{((x-x_c)^2+(y-y_c)^2+(z-z_c)^2)}$

$x,y: 10km - 50km$

$z: 0$

$G = 6.6743×10−11$

In [2]:
def model(G,m,x,y,z,x_c,y_c,z_c):
    return (G*m*(z-z_c))/((np.sqrt((x-x_c)**2+(y-y_c)**2+(z-z_c)**2))**3)

In [3]:
def jacobian(dx,dy,dz,dm):
    pass

In [4]:
G = 6.6743e-11
region = [10e3, 50e3, 10e3, 50e3]
x, y = bd.random_coordinates(region, size=1000, random_seed=42)
z = np.zeros_like(x)
m = 1e11
x_c = 25e3
y_c = 25e3
z_c = -15e3

g_z =  model(G,m,x,y,z,x_c,y_c,z_c)
g_z

array([6.23663262e-09, 2.64913957e-08, 5.45247101e-09, 9.80638733e-09,
       1.02446095e-08, 4.40209253e-09, 3.46251241e-09, 8.94517872e-09,
       8.44659542e-09, 2.59547330e-08, 1.49712311e-08, 4.48410516e-09,
       1.49071535e-08, 2.84083892e-09, 2.79406957e-08, 1.72854534e-08,
       1.14837430e-08, 5.02529134e-09, 5.05419505e-09, 1.60174390e-08,
       3.78157292e-09, 1.04801993e-08, 1.90814228e-09, 5.47991200e-09,
       6.55935311e-09, 2.07573782e-08, 4.75078257e-09, 1.24850098e-08,
       8.58202518e-09, 1.36173005e-08, 8.96525357e-09, 3.27880950e-09,
       2.42188236e-08, 5.67067560e-09, 2.65201541e-08, 1.25845488e-08,
       1.72617496e-08, 2.64318058e-08, 2.29653102e-08, 1.00354066e-08,
       1.46830326e-08, 5.44024398e-09, 8.43852091e-09, 1.44217149e-08,
       4.76475046e-09, 8.33991771e-09, 2.18954601e-08, 9.03383271e-09,
       4.83909767e-09, 1.59766647e-08, 1.20035285e-08, 8.02740581e-09,
       8.63038134e-09, 1.05298701e-08, 1.19366048e-08, 6.29918584e-09,
      

g